<a href="https://colab.research.google.com/github/vivekkishore/machinelearning/blob/main/querygpt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv
!pip install openai
!pip install --upgrade langchain
!pip install tiktoken
!pip install chromadb
import warnings
warnings.filterwarnings('ignore')
import langchain
langchain.debug=False
import os, openai
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())
openai.api_key=os.environ['API_KEY']

In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 11.8 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredExcelLoader
#Load
loader = UnstructuredExcelLoader("/content/Sample2.xlsx", mode="elements")
docs = loader.load()
docs[0]

# #Split
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10,separators=["\n\n", "\n", " ", ""])
# texts=text_splitter.split_documents(docs)



Document(page_content="\n\n\nAIR ID\nVendor Name\nApplication Status\nIntegration Specific\nIntegration Approach\nSP/IDP\nIntermediate screen present to enter UPN?\nAD Group Restriction\nSupported by LVA Team\n(Yes/No)\nRoles\nSupports Separate Stage & Production Environments\nSuperGroup\nYes/No\nGroup name vs Object ID\nStage AD Group/Stage SuperGroup Name\nProduction AD Group/Production SuperGroup Name\nProduction AD Group Owner/s\nAuto Provisioning\nName ID\nIf Name ID is not People Key. What is it & why?\nName ID format\nSupport for Prejoiners\nSupport for Contractors\nSupport for V&A\nMobile SSO\nmyL Asset - Yes/No\nIf Q= Yes, then Course Information\nADFS Relying Parties/Azure SSO Decommossioned\nDecommission RITMs\nVendor Contact\nBusiness Owner Contact\nVendor Server Hosting Location Country\nStage Cert Exp\nProd cert Exp\nStage Cert Exp Next 6 months\nProd Cert Exp Next 6 months\nStage Active / Inactive\n\n\n\n\n10040\nPluralsight\nActive\nMigrated to Azure AD\nSP\nNo\nYes, mu

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


from langchain.vectorstores import Chroma
persist_directory = '/content/chroma/'

# to remove old database files
# !rm -rf ./content/chroma/

db=Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)


In [ ]:
# #fetching relevant data directly from vector db using similarilty search
# Query='Is entity id required for all kinds of applications?'
# # result=db.similarity_search(Query, k=2)
# result1=db.max_marginal_relevance_search(Query,fetch_k=4,k=3)

from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0,openai_api_key= openai.api_key)

# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

# Run chain
from langchain.chains import RetrievalQA

def run_model(question):
  qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type='mmr',search_kwargs={'k': 3, 'fetch_k':4 }),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, verbose =True)
  result=qa_chain({"query":question})
  return result

In [ ]:
# including memory to store past conversation
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
from langchain.chains import ConversationalRetrievalChain

def run_memory_chain(question):
  qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=db.as_retriever(search_type='similarity',search_kwargs={'k': 3, 'fetch_k':4 }),
    chain_type="stuff",
    memory=memory,
    verbose=True)

  result =qa({"question":question})
  return result

